# Welcome to Jupyter on Chameleon, Bash style!

## Getting started

This document is similar in principle to the [Getting Started Guide][1], namely it shows how to
1. Create a baremetal server
* Access the baremetal server
* Destroy the baremetal server

but using the command line to complete the above steps. The primary application is to show how to automate your work by using scripts.

Before you do anything, you should first set up a few project variables. This is so the Jupyter Notebook knows which project to operate on and you can find the objects you created later. 

### Conventions

It is good practice to name the objects you create after your username. For instance, if your username is `bob`, your server should be named something like `bob_server`. The exact naming convention is defined by your group/project but it should still start with your username.

If you are only a member of one project, you can skip this step, as the project will be selected for you by default. You can also set the site you want to use via the `OS_REGION_NAME` setting - this defaults to `CHI@UC`.

[1]: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html

In [ ]:
# Set up user's project (Replace 'your-project' with your project name)
export OS_PROJECT_NAME='your-project'
# Set region (Optional, default to 'CHI@UC')
export OS_REGION_NAME='CHI@UC'


## Creating and launching a baremetal server

Starting a new baremetal server requires a few easy steps (we will cover the commands used in detail in another doc):
1. **Create a lease:** Specifically, we will create the lease `$USER-default-lease`. Now, some commands can use this lease name but others need a lease ID instead. So, while we are here we might as well get the `lease_id`.


In [ ]:
# Create a lease (instance reservation) named '$USER-default-lease' for one single node. 
# Change the name to fit your needs.
blazar lease-create --physical-reservation \
min=1,max=1,resource_properties='["=", "$node_type", "compute_haswell"]' "$USER-default-lease"

# Wait until lease is ready before continuing. 
# Another way is to keep track of PID and wait until it is done
lease_status=""
while [[ $lease_status != "ACTIVE" ]]
do
   sleep 5
   lease_status=$(blazar lease-show --format value -c status "$USER-default-lease")
done

echo "Lease $USER-default-lease ready for business"

# Get the lease/reservation ID associated with the lease '$USER-default-lease'
lease_id=$(blazar lease-show  --format value -c  reservations "$USER-default-lease" |grep \"id\"| cut -d \" -f4)


2. **Get the ID of the network we will create the baremetal server on:** In the previous step we obtained `lease_id`, the ID of the lease named `$USER-default-lease`. Now we will do the same but for the network we will run our server in. Use the `sharednet1` network unless you have a good reason not to. Further information on this network is [available in the docs][1].

[1]: https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_basic.html#shared-network

In [ ]:
# Get the network ID associated with sharednet1
network_id=$(openstack network show --format value -c id sharednet1)

3. **Create a SSH key pair:** One of the goals for this document is to access the the baremetal server; that will be achieved by using ssh to connect to the server. For [security][1], servers created in Chameleon are by default accessed using SSH key pair authentication. 

Jupyter created a key pair, 
* `~/.ssh/id_rsa`
* `~/.ssh/id_rsa.pub`

when your account was created. In this document we will use this default key. As with the lease, we do need a [name][2] associated with this key pair:

[1]:https://docs.openstack.org/horizon/latest/user/configure-access-and-security-for-instances.html
[2]:https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/keypair.html

In [ ]:
openstack keypair create --public-key ~/.ssh/id_rsa.pub  $USER-first-key

4. **Create the baremetal server:** 


In [ ]:
# Launch a Centos 7 baremetal server called '$USER-first-server' on network sharednet1 
# and lease '$USER-default-lease'. It will be accessible using Jupyter-created key pair.
openstack server create \
--flavor baremetal \
--image CC-CentOS7 \
--nic net-id="$network_id" \
--hint reservation="$lease_id" \
--key-name="$USER-first-key" \
--wait \
$USER-first-server

## Connecting to your server

By default, the server will only have a private IP assigned (in this case, an IP in the `sharednet1` network). In order to connect (using SSH or other protocol) to the server from your desktop or another computer in the internet, you should assign a [public-facing floating IP][1]. There are a limited amount of public IPs available across the entire Chameleon testbed, so try to keep the amount of nodes with a public IP to a minimum! A common practice is to set up one node as a "login node" with a public IP, and logging in to that node to manage all of your project's nodes.

### But wait! There's more!

Creating a server (or node) is not an instantanous process specially if it is a baremetal node. Chameleon has to boot the node, install the OS, move it to the right network, and then it is ready to receive the public IP. All of these steps can take **up to 10 minutes**, which is why we suggest to script building these nodes; you can let the script monitor the status of the node and then let you know when it is ready to be used. Or, same script can start using said node on its own. This is the reason why we passed the `--wait` flag when we created the server in the previous step: it will not allow the script to continue until the server is ready for the next step (adding the public IP).

### Steps:

1. Only continue when the server status = `ACTIVE`. `BUILD != ACTIVE` 
* Get a public floating IP. Either you already have a list of IPs you can use with your project or you will need to [request an IP][2]. For this example we will do the later. You need to know the name of the public network, which in this case is 'public'.
* Assign the IP to the server we created earlier.
* Ensure that there is a firewall rule or security policy allowing you to connect to server on port 22 (ssh). This is the default.
* Set up a remote connection to the node.

[1]: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html#associating-an-ip-address
[2]: https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/floating-ip.html#floating-ip-create


In [ ]:
# Wait until server is ready before continuing
server_status=""
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   server_status=$(openstack server list --format value -c Status --name "$USER-first-server")
done

# Request a public floating IP (in the 'public' network)
server_ip=$(openstack floating ip create public --format value -c floating_ip_address)

# Assign a public floating IP to $USER-first-server
openstack server add floating ip "$USER-first-server" "$server_ip"

## Testing the connection
Server has now a publicly facing IP. Let's see if it works.

In [ ]:
# Check if we can connect to server on port 22.
nc -q 0 -w 1 "${server_ip}" 22 < /dev/null &> /dev/null && echo "Up" || echo "Down"

The above script returns "Up" if it can connect to port 22 (SSH) on our recently created server using the public IP, otherwise it returns "Down". Of course, the answer can be changed so it is is more script-friendly.

## Logging into the server we just created

We ssh using the private key to verify it works. The default username is `cc`. While logged in, might as well take a quick look; remember this server will be wiped once we are done.

In [ ]:
ssh -i ~/.ssh/id_rsa cc@"$server_ip"

## Cleaning up after ourselves
As the last task in this document, tear everything down. We can put it all back together using scripts later.

In [ ]:
# Delete server
openstack server delete "$USER-first-server"

# Delete the public facing IP
openstack floating ip delete $server_ip

# Delete lease
blazar lease-delete "$USER-default-lease"

# Delete key (files are not deleted, just the "Name")
openstack keypair delete "$USER-first-key"